In [2]:
import sys
sys.path.append("C:/code/qdc2")
import numpy as np, matplotlib.pyplot as plt
from qdc.mmf.many_wl_fiber import ManyWavelengthFiber
from qdc.mmf.qdc_mmf_experiment import QDCMMFExperiment

# Short illustration of phase matching induced mode mixing 

In [2]:
def make_exp(M, is_step_index=False):
    mwf = ManyWavelengthFiber(N_wl=3, npoints=2**8, is_step_index=is_step_index)
    exp = QDCMMFExperiment(mwf)
    exp.set_phase_matching(Lc_um=4000, pump_waist_crystal=500,
                           magnification=M, wl_pump=0.405, n_pump=1.692)
    return mwf, exp

def run_and_collect(M, is_step_index=False):
    mwf, exp = make_exp(M, is_step_index)
    mid = len(mwf.fibers) // 2
    f = mwf.fibers[mid]
    f.set_input_gaussian(*exp.g_params)

    E0 = f.propagate(show=False)
    E_filt = exp._apply_phase_matching(E0)

    n = exp.n
    G = exp._pm_pump_amp if exp._pm_pump_amp is not None else np.ones((n, n))
    S = np.fft.fftshift(exp._pm_filter)

    F0 = np.abs(np.fft.fftshift(np.fft.fft2(E0.reshape(n, n))))
    F1 = np.abs(np.fft.fftshift(np.fft.fft2(E_filt.reshape(n, n))))
    I0 = np.abs(E0.reshape(n, n)) ** 2
    I1 = np.abs(E_filt.reshape(n, n)) ** 2
    return dict(G=G, S=S, I0=I0, I1=I1, F0=F0, F1=F1, dx=mwf.dx)

# -------- plot helper with colorbar --------
def show(ax, img, ttl, dx=None, fiber_radius_um=None):
    extent = None
    if dx is not None:
        n = img.shape[0]
        extent = [-(n // 2) * dx, (n // 2) * dx,
                  -(n // 2) * dx, (n // 2) * dx]

    im = ax.imshow(img, extent=extent)
    ax.set_title(ttl, fontsize=8)
    # ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.045, pad=0.02)

    # Add fiber core circle in real space
    if fiber_radius_um is not None and dx is not None:
        from matplotlib.patches import Circle
        circle = Circle((0, 0), radius=fiber_radius_um, edgecolor='red',
                        facecolor='none', lw=1)
        ax.add_patch(circle)

Ms = [3.3, 10]
is_step_index = True
data1 = run_and_collect(M=Ms[0], is_step_index=is_step_index)
data2 = run_and_collect(M=Ms[1], is_step_index=is_step_index)

titles = ['Gaussian', 'sinc PM', 'real |E|² before', 'real |E|² after',
          '|FFT| before', '|FFT| after']
rows = ['G', 'S', 'I0', 'I1', 'F0', 'F1']

fig, axes = plt.subplots(len(rows), 2, figsize=(9, 14))
for col, data, M in zip([0, 1], [data1, data2], Ms):
    dx = data['dx']
    for r, key in enumerate(rows):
        is_real_space = key in ['G', 'I0', 'I1']
        show(axes[r, col], data[key], f'{titles[r]} (M={M})',
             dx=dx if is_real_space else None,
             fiber_radius_um=25 if is_real_space else None)

plt.tight_layout()
plt.show()


Getting 3 fibers...


0it [00:00, ?it/s]C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\pyMMF\solvers\SI.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return jv(m, u) / (u * jv(m - 1, u)) + kn(m, w) / (w * kn(m - 1, w))
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\pyMMF\solvers\SI.py:147: RuntimeWarning: invalid value encountered in divide
  return jv(m, u) / (u * jv(m - 1, u)) + kn(m, w) / (w * kn(m - 1, w))
3it [00:31, 10.36s/it]


Got fibers!
Getting 3 fibers...


3it [00:01,  2.04it/s]


Got fibers!


# Illustration of different $\partial\beta_m / \partial\omega $ for different $m$ values 

In [16]:
import sys
sys.path.append("C:/code/qdc2")
import numpy as np, matplotlib.pyplot as plt
from qdc.mmf.many_wl_fiber import ManyWavelengthFiber

# SI
is_step_index = True
mwf_SI = ManyWavelengthFiber(wl0=0.810, Dwl=0.002, N_wl=3, npoints=2**8, is_step_index=is_step_index)
mwf_SI.betas = mwf_SI.betas * 1e6 # convert to 1/m
cutoff_SI = 200 
print(f'{mwf_SI.betas.max()=}, {mwf_SI.betas.min()=}')

# GRIN 
is_step_index = False
mwf_GRIN = ManyWavelengthFiber(wl0=0.810, Dwl=0.002, N_wl=3, npoints=2**8, is_step_index=is_step_index)
mwf_GRIN.betas = mwf_GRIN.betas * 1e6 # convert to 1/m
cutoff_GRIN = 100 
print(f'{mwf_GRIN.betas.max()=}, {mwf_GRIN.betas.min()=}')


fig, ax = plt.subplots(2, 2, figsize=(11, 6), constrained_layout=True)

diffs_SI = np.diff(mwf_SI.betas, axis=0)  
diffs_SI = diffs_SI[:, :cutoff_SI] 
ax[0, 0].plot(range(len(diffs_SI[0])), diffs_SI[0], '.-', label=r'$\delta\beta_1$')
# ax[0, 0].plot(range(len(diffs_SI[1])), diffs_SI[1], 'o-', label=r'$\delta\beta_2$')
# ax[0, 0].legend()
ax[0, 0].set_ylabel(r'$\delta\beta$ (1/m)') # Units? 
ax[0, 0].set_title(rf'$\delta\beta$ step index')
ax[0, 0].set_xlabel(r'mode number')  

ddiff_SI = diffs_SI[1] - diffs_SI[0]
ax[1, 0].plot(range(len(ddiff_SI)), ddiff_SI, '.-')
ax[1, 0].set_title(rf'$\delta^2\beta$ step index')
ax[1, 0].set_ylabel(r'$\delta^2\beta$ (1/m)') # Units? 
ax[1, 0].set_xlabel(r'mode number')  

diffs_GRIN = np.diff(mwf_GRIN.betas, axis=0)  
diffs_GRIN = diffs_GRIN[:, :cutoff_GRIN] 
ax[0, 1].plot(range(len(diffs_GRIN[0])), diffs_GRIN[0], '.-', label=r'$\delta\beta_1$')
# ax[0, 1].plot(range(len(diffs_GRIN[1])), diffs_GRIN[1], 'o-', label=r'$\delta\beta_2$')
# ax[0, 1].legend()
ax[0, 1].set_ylabel(r'$\delta\beta$ (1/m)') # Units? 
ax[0, 1].set_title(rf'$\delta\beta$ graded index')
ax[0, 1].set_xlabel(r'mode number')  

ddiff_GRIN = diffs_GRIN[1] - diffs_GRIN[0]
ax[1, 1].plot(range(len(ddiff_GRIN)), ddiff_GRIN, '.-')
ax[1, 1].set_xlabel(r'mode number')  
ax[1, 1].set_ylabel(r'$\delta^2\beta$ (1/m)') # Units? 
# ax[1, 1].set_title(rf'graded index')
ax[1, 1].set_title(rf'$\delta^2\beta$ graded index')

ax[0, 0].text(-0.1, 1.15, '(a)', color='black', fontsize=14, fontweight='bold', transform=ax[0, 0].transAxes, va='top', ha='left')
ax[0, 1].text(-0.1, 1.15, '(b)', color='black', fontsize=14, fontweight='bold', transform=ax[0, 1].transAxes, va='top', ha='left')
ax[1, 0].text(-0.1, 1.15, '(c)', color='black', fontsize=14, fontweight='bold', transform=ax[1, 0].transAxes, va='top', ha='left')
ax[1, 1].text(-0.1, 1.15, '(d)', color='black', fontsize=14, fontweight='bold', transform=ax[1, 1].transAxes, va='top', ha='left')


fig.show()
fig.savefig(fr'G:\My Drive\Projects\Dispersion Cancelation\Paper\Final Figures\beta_tag.png')

if True:
    fig, ax = plt.subplots(2, 1, figsize=(11, 6), constrained_layout=True)
    ax[0].plot(range(len(mwf_SI.betas[0])), mwf_SI.betas[0], '.-', label=f'lambda={mwf_SI.wls[0]:.5f}')
    ax[0].plot(range(len(mwf_SI.betas[1])), mwf_SI.betas[1], '.-', label=f'lambda={mwf_SI.wls[1]:.5f}')
    ax[0].plot(range(len(mwf_SI.betas[2])), mwf_SI.betas[2], '.-', label=f'lambda={mwf_SI.wls[2]:.5f}')
    ax[0].legend()
    ax[0].set_title(rf'$\beta$ step index')
    ax[1].plot(range(len(mwf_GRIN.betas[0])), mwf_GRIN.betas[0], '.-', label=f'lambda={mwf_GRIN.wls[0]:.5f}')
    ax[1].plot(range(len(mwf_GRIN.betas[1])), mwf_GRIN.betas[1], '.-', label=f'lambda={mwf_GRIN.wls[1]:.5f}')
    ax[1].plot(range(len(mwf_GRIN.betas[2])), mwf_GRIN.betas[2], '.-', label=f'lambda={mwf_GRIN.wls[2]:.5f}')
    ax[1].set_title(rf'$\beta$ graded index')
    ax[1].legend()
    ax[1].set_xlabel(r'mode number')
    fig.show()


Getting 3 fibers...


3it [00:01,  2.14it/s]


Got fibers!
mwf_SI.betas.max()=11382684.554011125, mwf_SI.betas.min()=11258308.710157316
Getting 3 fibers...


3it [00:00,  3.62it/s]


Got fibers!
mwf_GRIN.betas.max()=11377617.235149734, mwf_GRIN.betas.min()=11260067.904145923


In [25]:
fig, ax = plt.subplots(figsize=(10, 5))
imm = ax.plot(sorted(mwf_SI.betas[0, :]), '.-')
fig.show()

fig, ax = plt.subplots(figsize=(10, 5))
imm = ax.plot(sorted(mwf_GRIN.betas[0, :]), '.-')
fig.show()


In [38]:
print(mwf_SI.wls)
print(mwf_SI.betas[:, 0])
print(diffs_SI[:, 0])
print('--------------')
print(mwf_GRIN.wls)
print(mwf_GRIN.betas[:, 0])
print(diffs_GRIN[:, 0])

[0.8096668  0.81033347 0.81100124]
[11382684.55401113 11373231.97496347 11363779.5248688 ]
[-9452.57904765 -9452.45009467]
--------------
[0.8096668  0.81033347 0.81100124]
[11377617.23514973 11368164.92113158 11358712.77676263]
[-9452.31401815 -9452.14436896]


In [ ]:
# ax[0].legend()
# ax[0].set_title('betas (1/m)')

# c = 299792458
# omegas = 2*np.pi*c/mwf_SI.wls
# Domega = omegas[1] - omegas[0]
# after 1m, d_beta/d_omega * Delta_omega * L, and I want to see how this differs between different modes, 
# L = 0.1e6

# MMF speckle envelope to normalize before PCC 

In [ ]:
import sys
sys.path.append("C:/code/qdc2")
import numpy as np, matplotlib.pyplot as plt
from qdc.mmf.many_wl_fiber import ManyWavelengthFiber
from qdc.mmf.fiber import Fiber

is_step_index = True

fiber_L = 0.2e6  if is_step_index else 3e6 # um 
N_wl = 81
N_classical = 5
N_SPDC = 5
wl0 = 0.810
Dwl = 0.010 if is_step_index else 0.020
NA_ref = 0.2   
dzs = [0, 10, 50, 200, 1000] if is_step_index else [0, 10, 50, 100]
npoints = 2**7
free_mode_matrix = False if npoints == 2**7 else True  # when working with 2**8, the RAM explodes 
autosolve = not free_mode_matrix  # if freeing each time - no point in autosolving initially 


n_pixels_diameter = npoints//5 if is_step_index else npoints//5
s = ManyWavelengthFiber(wl0=wl0, Dwl=Dwl, N_wl=N_wl, fiber_L=fiber_L, rng_seed=42, is_step_index=is_step_index, 
                        npoints=npoints, NA_ref=NA_ref, autosolve=autosolve)
s.gaussian_params = np.array([2.7, 7, 10, 0.5, 0.5]) 
s.gaussian_dparams = np.array([1, 5, 5, 0.3, 0.3])

f = s.fibers[0]

In [17]:
f.set_input_gaussian(*mwf.get_g_params())
f.propagate(show=False)
I_out = np.abs(f.profile_end)**2

for i in range(100):
    if i % 10 == 0:
        print(i)
    f.set_input_gaussian(*mwf.get_g_params())
    f.propagate(show=False)
    I_out += np.abs(f.profile_end)**2

fig, ax = plt.subplots(figsize=(10, 5))
imm = ax.imshow(I_out.reshape(f.npoints, f.npoints))
fig.colorbar(imm, ax=ax)
fig.show()

0
10
20
30
40
50
60
70
80
90


# Free-space causes mode-mixing

In [3]:
from qdc.mmf.fiber import Fiber
from qdc.mmf.qdc_mmf_experiment import propagate_free_space
is_step_index = True
f = Fiber(is_step_index=is_step_index, npoints=2**8, diameter=50, areaSize=80)

In [15]:
# Visulaize for some mode 
mode_no = 100
dz = 500 # um 
n = f.npoints
mode = f.modes.getModeMatrix()[:, mode_no].reshape((n, n))
after_propagation = propagate_free_space(mode, dz, f.wl, f.dx)
modes_after = f.modes.getModeMatrix().T @ after_propagation
f.show_profile(mode)
f.show_profile(after_propagation)
plt.show()

In [ ]:
dz = 500 # um 
n = f.npoints
modes_to_check = f.Nmodes
# modes_to_check = 50
modal_TM = np.zeros((modes_to_check, f.Nmodes), dtype=complex)
for mode_no in range(modes_to_check):
    if mode_no % 10 == 0:
        print(mode_no)
    mode = f.modes.getModeMatrix()[:, mode_no].reshape((n, n))
    after_propagation = propagate_free_space(mode, dz, f.wl, f.dx)
    modes_after = f.modes.getModeMatrix().T @ after_propagation
    modal_TM[mode_no, :] = modes_after

In [24]:
fig, ax = plt.subplots(figsize=(10, 5))
imm = ax.imshow(np.abs(modal_TM)**2, vmax=0.03)
ax.set_title(f'{is_step_index=}; {dz=} um')
fig.colorbar(imm)
fig.show()

In [96]:
(np.abs(modal_TM)**2).sum(axis=1)

array([0.97238507, 0.8253959 , 0.5707424 , 0.45091823, 0.23735816,
       0.10689055, 0.05272959, 0.0322103 , 0.0240018 , 0.02093807,
       0.02125135, 0.0268941 , 0.07734683, 0.91270644, 0.68278661,
       0.49330301, 0.34552393, 0.17309375, 0.07506526, 0.03873544,
       0.02557374, 0.02044269, 0.01902472, 0.02096998, 0.03220226,
       0.91270644, 0.68278661, 0.49330301, 0.34552393, 0.17309375,
       0.07506525, 0.03873544, 0.02557373, 0.02044269, 0.01902472,
       0.02096999, 0.03220226, 0.85093924, 0.57681275, 0.44628512,
       0.23626059, 0.10848458, 0.05353391, 0.03247343, 0.02402267,
       0.02081929, 0.0209787 , 0.02622222, 0.06935756, 0.85076344])

# FOO

In [1]:
import sys
sys.path.append("C:/code/qdc2")
import numpy as np, matplotlib.pyplot as plt
from qdc.mmf.many_wl_fiber import ManyWavelengthFiber
from qdc.mmf.fiber import Fiber

# SI
is_step_index = True
mwf_SI = ManyWavelengthFiber(wl0=0.810, Dwl=0.002, N_wl=3, npoints=2**8, is_step_index=is_step_index)
mwf_SI.betas = mwf_SI.betas * 1e6 # convert to 1/m

# GRIN 
is_step_index = False
mwf_GRIN = ManyWavelengthFiber(wl0=0.810, Dwl=0.002, N_wl=3, npoints=2**8, is_step_index=is_step_index)

is_step_index = False 
f = Fiber(is_step_index=is_step_index, npoints=2**8, diameter=50, areaSize=80, wl=0.720)

Getting 3 fibers...


0it [00:00, ?it/s]C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\pyMMF\solvers\SI.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return jv(m, u) / (u * jv(m - 1, u)) + kn(m, w) / (w * kn(m - 1, w))
C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\pyMMF\solvers\SI.py:147: RuntimeWarning: invalid value encountered in divide
  return jv(m, u) / (u * jv(m - 1, u)) + kn(m, w) / (w * kn(m - 1, w))
3it [00:33, 11.12s/it]


Got fibers!
Getting 3 fibers...


3it [00:05,  2.00s/it]


Got fibers!


In [ ]:

envelope = (np.abs(mwf_GRIN.fibers[0].modes.getModeMatrix())**2).sum(axis=1)
# fig, ax = plt.subplots(figsize=(10, 5))
# imm = ax.imshow(envelope.reshape(f.npoints, f.npoints))
# fig.colorbar(imm, ax=ax)
# fig.show()

In [ ]:
.modes.getModeMatrix().shape

(65536, 383)

In [ ]:
f.modes.l